<table width="100%" border="3">
  <tbody>
    <tr>
      <td ><img src="https://aidea-web.tw/images/web/logo_white.png" alt="Aidea" width="400"/></td>
      <td align='left'><h1>Exercise 4: Ensemble Learning </h1></td>
    </tr>
  </tbody>
</table>

# AIdea AOI 實作課程, 2020
* 這個教程使用工研院AIdea人工智慧共創平台的AOI資料集做為練習的標的。
* 介紹撰寫深度學習的程式來進行自動光學檢查的瑕疵分類。
* 此notebook程式可以在雲端使用Google Colab或使用個人電腦上的Jupyter執行。

AIdea人工智慧共創平台
https://aidea-web.tw/topic/a49e3f76-69c9-4a4a-bcfc-c882840b3f27

亞洲大學 朱學亭老師  EMAIL: htchu.taiwan@gmail.com  FB: https://www.facebook.com/htchu.taiwan

# Exercise 4: Ensemble method
https://en.wikipedia.org/wiki/Ensemble_learning
<img src="https://upload.wikimedia.org/wikipedia/commons/d/de/Stacking.png">

## Step 1: Load AIdea AOI dataset

In [ ]:
#load AIdea AOI dataset
from google_drive_downloader import GoogleDriveDownloader
GoogleDriveDownloader.download_file_from_google_drive(file_id='1tovCO2gsjesjJ8OsfHgahyt-buY34dk0',dest_path='./content', unzip=True)

Unzipping...Done.


## Step 2: Import python libraries

In [ ]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.2.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Input, Concatenate
from tensorflow.keras.layers import Dropout, Flatten, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

## Step 3: Load pretrained models:
* m1: auaoi-InceptionResNetV2
* m2: auaoi-InceptionV3
* m3: auaoi-Xception

In [ ]:
#load AIdea AOI trained CNN models
from google_drive_downloader import GoogleDriveDownloader
GoogleDriveDownloader.download_file_from_google_drive(file_id='1sh735v1fkLU3EC-xZCIq6mmvYkKyRUq9',dest_path='./model', unzip=True)

Unzipping...Done.


In [ ]:
model_files = ['m1.h5', 'm2.h5', 'm3.h5']
models = []
for file in model_files:
  model = tf.keras.models.load_model(file)
  models.append(model)


In [ ]:
for model in models:
  print(model.name)

inception_resnet_v2
inception_v3
xception


## Step 4: read the training set

In [ ]:
import pandas as pd
df_train = pd.read_csv("train.csv",dtype=str)
print(df_train.shape)

(2528, 2)


In [ ]:
df_train.head()

,ID,Label
0,train_00000.png,0
1,train_00001.png,1
2,train_00002.png,1
3,train_00003.png,5
4,train_00004.png,5


In [ ]:
train_num = df_train.shape[0]
train_files = df_train.iloc[:train_num,0].values
train_labels = df_train.iloc[:train_num,1].values
print(train_labels[:20])

['0' '1' '1' '5' '5' '5' '3' '0' '3' '5' '3' '5' '3' '3' '1' '1' '1' '1'
 '5' '1']


In [ ]:
import seaborn as sns
g = sns.countplot(train_labels)

In [ ]:
num_classes=6

## Step 5: Set up an ImageDataGenerator and a Train_generator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input
#from tensorflow.keras.applications.inception_v3 import preprocess_input
img_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_generator = img_gen.flow_from_dataframe(dataframe=df_train,
            directory="train_images",
            x_col="ID",
            y_col="Label",
            subset=None,
            batch_size=8,
            shuffle=False,
            class_mode="categorical",
            color_mode="rgb",
            target_size=(299,299))
print(train_generator.n)

Found 2528 validated image filenames belonging to 6 classes.


In [ ]:
train_generator.class_indices

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}

In [ ]:
if train_generator.n % train_generator.batch_size ==0:
  step_size_train=train_generator.n//train_generator.batch_size
else:
  step_size_train=train_generator.n//train_generator.batch_size + 1
print(step_size_train)

## Step 6: Evaluation of pretrained models with the training set

In [ ]:
##Evaluation of pretrained models
for model in models:
    train_generator.reset()
    model.evaluate_generator(generator=train_generator, steps=step_size_train, verbose=1)

Instructions for updating:
Please use Model.evaluate, which supports generators.
316/316 [==============================] - 47s 148ms/step - loss: 0.0069 - accuracy: 0.9980


## Step 7: Prediction of pretrained models with the training set

In [ ]:
#y_predictions = model.predict(X_train, batch_size=20)
y_predictions_All = []
for model in models:
    train_generator.reset()
    y_predictions = model.predict_generator(generator=train_generator, steps=step_size_train, verbose=1)
    y_predictions_All.append(y_predictions)

Instructions for updating:
Please use Model.predict, which supports generators.
316/316 [==============================] - 46s 145ms/step


## Step 8: Confusion matrices of pretrained models with the training set

In [ ]:
labels = train_labels.astype(int)
print(labels[:10])

[0 1 1 5 5 5 3 0 3 5]


In [ ]:
from sklearn.metrics import confusion_matrix
predicts_all = []
for y_predictions in y_predictions_All:
    predicts = np.argmax(y_predictions,axis=1)
    predicts_all.append(predicts)
    confusion=confusion_matrix(labels, predicts)
    print(confusion)

[[674   0   0   0   0   0]
 [  5 482   0   3   1   1]
 [  1   0  99   0   0   0]
 [  0   0   0 376   2   0]
 [  0   0   0   2 238   0]
 [  0   0   0   0   0 644]]
[[674   0   0   0   0   0]
 [  1 490   0   0   1   0]
 [  0   0 100   0   0   0]
 [  0   0   0 377   1   0]
 [  0   0   0   1 239   0]
 [  0   2   0   0   0 642]]
[[674   0   0   0   0   0]
 [  1 490   0   1   0   0]
 [  0   0 100   0   0   0]
 [  0   0   0 377   1   0]
 [  0   0   0   2 238   0]
 [  0   0   0   0   0 644]]


## Step 9: Show differences of predictions between the models

In [ ]:
for i in range(len(labels)):
    label=labels[i]
    pred0=predicts_all[0][i]
    pred1=predicts_all[1][i]
    pred2=predicts_all[2][i]
    if label!=pred0 or label!=pred1 or label!=pred2:
        print(f'{label}->({pred0}, {pred1}, {pred2})')

4->(4, 4, 3)
1->(1, 0, 1)
1->(0, 1, 0)
4->(3, 3, 3)
1->(5, 1, 1)
5->(5, 1, 5)
1->(0, 1, 1)
3->(4, 3, 3)
1->(0, 1, 1)
3->(4, 3, 3)
1->(3, 1, 3)
1->(3, 1, 1)
1->(3, 1, 1)
4->(3, 4, 4)
1->(0, 1, 1)
1->(1, 4, 1)
5->(5, 1, 5)
3->(3, 4, 3)
1->(0, 1, 1)
1->(4, 1, 1)
2->(0, 2, 2)
3->(3, 3, 4)


## Step 10: Ensemble predictions by indiviual models

In [ ]:
y_predictions_ensemble = y_predictions_All[0]+y_predictions_All[1]+y_predictions_All[2]
print(y_predictions_ensemble[:2])
type(y_predictions)

[[2.9977081e+00 2.6643804e-05 2.0590229e-03 2.0168223e-04 4.4715466e-06
  3.2687109e-09]
 [4.1103329e-11 2.9999866e+00 1.3372697e-05 3.4153191e-10 7.0617383e-09
  1.2877229e-08]]


numpy.ndarray

In [ ]:
predicts_ensemble = np.argmax(y_predictions_ensemble,axis=1)
print(predicts_ensemble[0:10])

[0 1 1 5 5 5 3 0 3 5]


## Step 11: Show differences of predictions by the ensembled models

In [ ]:
for i in range(len(labels)):
    label=labels[i]
    pred0=predicts_all[0][i]
    pred1=predicts_all[1][i]
    pred2=predicts_all[2][i]
    predx=predicts_ensemble[i]
    if label!=pred0 or label!=pred1 or label!=pred2:
        print(f'{label}->({pred0}, {pred1}, {pred2})=>{predx}')

4->(4, 4, 3)=>4
1->(1, 0, 1)=>1
1->(0, 1, 0)=>0
4->(3, 3, 3)=>3
1->(5, 1, 1)=>1
5->(5, 1, 5)=>5
1->(0, 1, 1)=>1
3->(4, 3, 3)=>3
1->(0, 1, 1)=>1
3->(4, 3, 3)=>3
1->(3, 1, 3)=>3
1->(3, 1, 1)=>1
1->(3, 1, 1)=>1
4->(3, 4, 4)=>4
1->(0, 1, 1)=>1
1->(1, 4, 1)=>1
5->(5, 1, 5)=>5
3->(3, 4, 3)=>3
1->(0, 1, 1)=>1
1->(4, 1, 1)=>1
2->(0, 2, 2)=>2
3->(3, 3, 4)=>3


In [ ]:
confusion=confusion_matrix(labels, predicts_ensemble)
print(confusion)

[[674   0   0   0   0   0]
 [  1 490   0   1   0   0]
 [  0   0 100   0   0   0]
 [  0   0   0 378   0   0]
 [  0   0   0   1 239   0]
 [  0   0   0   0   0 644]]


In [ ]:
overkill= []
underkill = []
for i in range(train_num):
  if labels[i] == 0 and predicts_ensemble[i] !=0:
    overkill.append(i)
  if labels[i] != 0 and predicts_ensemble[i] ==0:
    underkill.append(i)
print('# of overkill= {}; # of underkill= {} '.format(len(overkill), len(underkill)))

# of overkill= 0; # of underkill= 1 


## Step 12: Preparing test images

In [ ]:
df_test = pd.read_csv("test.csv",dtype=str)
print(df_test.shape)

(10142, 2)


In [ ]:
df_test.head()

In [ ]:
test_num = df_test.shape[0]
test_files  = df_test.iloc[:test_num,0].values
test_labels = df_test.iloc[:test_num,1].values
print(test_labels[:10])

[nan nan nan nan nan nan nan nan nan nan]


In [ ]:
img_gen2 = ImageDataGenerator(horizontal_flip=False, vertical_flip=False, preprocessing_function=preprocess_input)
test_generator = img_gen2.flow_from_dataframe(dataframe=df_test,
            directory="test_images",
            x_col="ID",
            y_col="Label",
            batch_size=32,
            shuffle=False,
            class_mode=None,
            target_size=(299,299))

Found 10142 validated image filenames.


In [ ]:
if test_generator.n % test_generator.batch_size ==0:
  step_size_test=test_generator.n//test_generator.batch_size
else:
  step_size_test=test_generator.n//test_generator.batch_size + 1
print(step_size_test)

317


## Step 13: Prediction of pretrained models with the testing set

In [ ]:
#y_predictions = model.predict(X_train, batch_size=20)
y_predictions_All = []
for model in models:
    test_generator.reset()
    y_predictions = model.predict_generator(generator=test_generator, steps=step_size_test,verbose=1)
    y_predictions_All.append(y_predictions)

317/317 [==============================] - 168s 529ms/step


## Step 14: Ensemble Predictions

In [ ]:
y_predictions_ensemble = y_predictions_All[0]+y_predictions_All[1]+y_predictions_All[2]
y_predictions_ensemble.shape

(10142, 6)

In [ ]:
predicts_ensemble=np.argmax(y_predictions_ensemble,axis=1)

## Step 15: Output the predictions

In [ ]:
df_out = pd.DataFrame(df_test)
df_out.shape

(10142, 2)

In [ ]:
df_out['Label'] = predicts_ensemble
df_out.to_csv("0627-ensemble.csv", index=False)